In [1]:
!which python

/Users/pushkalm11/miniconda3/envs/unrolling/bin/python


In [2]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import linalg as LA
from numpy.linalg import matrix_power as MP

In [3]:
import random
import scipy
from scipy.spatial.distance import cdist
from scipy.io import savemat, loadmat

In [4]:
import networkx as nx
import sys

In [5]:
import tensorflow as tf
from tensorflow.keras import initializers
from tensorflow.keras.layers import Flatten, Dense
from keras import backend as K
from keras.callbacks import EarlyStopping

In [6]:
print(f"TF Version: {tf.__version__}")

TF Version: 2.14.0


In [7]:
X_train = loadmat('./Dataset/X_train.mat')['X_Ground']
Adj_matrix = loadmat('./Dataset/A_matic.mat')['A_learned']

N, T = X_train.shape

In [8]:
dt = np.zeros([T - 1, T])

for i in range(T - 1):
    dt[i, i] = -1
    dt[i, i - 1] = 1

d = dt.T
d = np.concatenate((d, np.zeros([T, 1])), axis = 1) 
d[T - 1, T - 1] = -1
d = d.T
d[T - 1, T - 1] = 1 
d[-1, -2] = 1
d[0, -1] = 0
d = tf.convert_to_tensor(d, dtype = "float32")
print(d)

tf.Tensor(
[[-1.  0.  0. ...  0.  0.  0.]
 [ 1. -1.  0. ...  0.  0.  0.]
 [ 0.  1. -1. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ... -1.  0.  0.]
 [ 0.  0.  0. ...  1. -1.  0.]
 [ 0.  0.  0. ...  0.  1.  1.]], shape=(500, 500), dtype=float32)


In [9]:
def adj_to_laplacian(AdjL):
    tmp = np.array(AdjL, dtype = "float32")
    return np.diag(np.sum(tmp, axis = 0)) - tmp

In [10]:
psi_cost = 0
test_list = []

In [11]:
# A - Predicted, B - Target matrix
def costfunc(A, B):
    # (y_true, y_pred)
    # print(A.shape)
    # print(B.shape)
    return (tf.norm(tf.multiply(psi_cost, (A - B)))) ** 2

# def costfunc(A, B):
#     return (tf.norm(tf.multiply(psi_cost, (A - B))) ** 2) + (reg * tf.norm(HLG @ B @ GLT) ** 2 / 2)

def mse_unknown(A, B):

    global psi_cost
    psi = psi_cost

    psid = tf.convert_to_tensor(np.ones([N, T], dtype = float) - psi, dtype = "float32")
    mse_unk = (LA.norm(np.multiply(psid, (X_train - B)))) ** 2 / np.sum(psid.numpy().flatten())
    
    global mse_unknown_list
    mse_unknown_list.append(mse_unk)
    
    return mse_unk

def mse_known(A, B):

    global psi_cost 
    psi = psi_cost

    mse_kn = (LA.norm(np.multiply(psi, (X_train - B)))) ** 2 / np.sum(psi.numpy().flatten())
    
    global mse_known_list
    mse_known_list.append(mse_kn)

    return mse_kn

In [12]:
callb = EarlyStopping(monitor = 'loss', mode = 'min', min_delta = 0.00001, verbose = 1, patience = 10)

def scheduler(epoch, lr):
    if epoch < 15:
        return lr
    else:
        return lr / lr_decay_factor

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [13]:
class EldarModel(tf.keras.Model):
    
    def __init__(self, Laplacian, reg_init, loops_init, d_init, **kwargs):
        super(EldarModel, self).__init__(**kwargs)
        p_init = 15.0
        self.p1_init = tf.constant(p_init, shape = (1, 1), dtype = "float32")
        self.p1 = tf.Variable(initial_value = self.p1_init, trainable = True, name = "p1")
        self.p2_init = tf.constant(p_init, shape = (1, 1), dtype = "float32")
        self.p2 = tf.Variable(initial_value = self.p2_init, trainable = True, name = "p2")
        self.p3_init = tf.constant(p_init, shape = (1, 1), dtype = "float32")
        self.p3 = tf.Variable(initial_value = self.p3_init, trainable = True, name = "p3")
        self.p4_init = tf.constant(p_init, shape = (1, 1), dtype = "float32")
        self.p4 = tf.Variable(initial_value = self.p4_init, trainable = True, name = "p4")
        
        self.lap = Laplacian
        self.reg = reg_init
        self.loops = loops_init
        self.d_matrix = d_init
        
    def call(self, y):
        
        psi = y[0, :, T:]
        global psi_cost
        psi_cost = psi
        
        psi = tf.convert_to_tensor(psi, dtype = "float32")
        
        X_in = y[0, :, : T]
        X_in = tf.convert_to_tensor(X_in, dtype = "float32")
        
        Y = X_in
        
        LG, reg, loop, d = self.lap, self.reg, self.loops, self.d_matrix
        
        LT = tf.matmul(tf.transpose(d), d)
        LT = tf.convert_to_tensor(LT, dtype = "float32")
        
        HLG = tf.eye(tf.shape(LG)[0], dtype = "float32") 
        GLT = LT + (self.p1 * MP(LT, 2)) + (self.p2 * MP(LT, 3)) + (self.p3 * MP(LT, 4)) + (self.p4 * MP(LT, 5))
        
        Xk = tf.zeros_like(X_in)
        Zk = tf.zeros_like(X_in)
        Zk = -(tf.multiply(psi, Xk) - Y + (reg * (HLG @ Xk @ GLT)))
        
        for i in range(loop):
            fdx_xk = tf.multiply(psi, Xk) - Y + (reg * (HLG @ Xk @ GLT))
            fdx_zk = tf.multiply(psi, Zk) - Y + (reg * (HLG @ Zk @ GLT))
            
            tau = tf.linalg.trace(tf.transpose(fdx_xk) @ Zk) / tf.linalg.trace(tf.transpose((Y + fdx_zk)) @ Zk)
            
            Xk_1 = Xk - (tau * Zk)
            
            fdx_xk_1 = tf.multiply(psi, Xk_1) - Y + (reg * (HLG @ Xk_1 @ GLT))
            
            gamma = (tf.norm(fdx_xk_1) ** 2) / (tf.norm(fdx_xk) ** 2)
            
            Zk_1 = (gamma * Zk) - fdx_xk_1
            
            Xk = Xk_1
            Zk = Zk_1
        
        global test_list
        test_list.append(Xk_1.numpy())
        
        return tf.reshape(Xk_1, [1, N, T])

In [14]:
no_of_psi = 10
Laplacian = adj_to_laplacian(Adj_matrix)

sensing_ratio = [0.01, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.4, 0.45, 0.5]

# reg_list = np.load("./Outputs/eldar_GT_reg.npy")
# loops_list = np.load("./Outputs/eldar_GT_loops.npy")

reg_list = [1.0e-2] * len(sensing_ratio)
loops_list = [50] * len(sensing_ratio)

all_psi = []

mse_known_list = []
mse_unknown_list = []

learned_variables = []

lr_decay_factor = 1.05

In [15]:
for i_sen, rem in enumerate(sensing_ratio):
    
    print(f"\n\n###########################################################################################")
    print(f"################################### Sensing Ratio: {rem * 100}% ###################################")
    print(f"###########################################################################################\n\n")
    
    if i_sen == 0:
        M = int(rem * T)
        X_train_missing = np.zeros([no_of_psi, N, T])
        X_train_concatenated = np.zeros([no_of_psi, N, 2*T])
    
        for i in range(no_of_psi):
            psi_k = np.array([0] * (N * M) + [1] * (N * (T - M)))
            np.random.shuffle(psi_k)
            psi_k = psi_k.reshape([N, T])
            all_psi.append(psi_k)
            X_train_missing[i, : , : ] = X_train * psi_k
            X_train_concatenated[i, :, :] = np.concatenate((X_train_missing[i], psi_k), axis = 1)
    
    else:
        previous_psi = all_psi[-no_of_psi : ]
        X_train_missing = np.zeros([no_of_psi, N, T])
        X_train_concatenated = np.zeros([no_of_psi, N, 2 * T])
        
        rem_diff = int(N * (rem - sensing_ratio[i_sen - 1]) * T)
        
        for i in range(no_of_psi):
            psi_k_tmp = np.array(previous_psi[i]).flatten()
            ones_idx = np.where(psi_k_tmp == 1)
            
            ones_to_zero_idx = np.random.choice(ones_idx[0], size = rem_diff)
            psi_k_tmp[ones_to_zero_idx] = 0
            psi_k = psi_k_tmp
            
            psi_k = psi_k.reshape([N, T])
            all_psi.append(psi_k)
            X_train_missing[i, : , : ] = X_train * psi_k
            X_train_concatenated[i, :, :] = np.concatenate((X_train_missing[i], psi_k), axis = 1)
        
    reg_init = reg_list[i_sen]
    loops_init = loops_list[i_sen]
    
    sgd_optimizer = tf.keras.optimizers.SGD(learning_rate = 1.0e-3, momentum = 0.0)
    
    eldar_model = EldarModel(Laplacian, reg_init, loops_init, d, name = 'EM')
    eldar_model.compile(optimizer = sgd_optimizer, loss = costfunc, 
                        metrics = [mse_known, mse_unknown], run_eagerly = True)
    eldar_model.fit(X_train_concatenated, X_train_missing, epochs = 100, 
                    callbacks = [lr_scheduler], batch_size = 1)
    eldar_model.summary()
    
    all_variables = eldar_model.variables
    
    graph_vars = [i.numpy()[0, 0] for i in all_variables]
    learned_variables.append(graph_vars)
    
    print(f"Learned Variables: {graph_vars}")



###########################################################################################
################################### Sensing Ratio: 1.0% ###################################
###########################################################################################


Epoch 1/100
10/10 [==============================] - 14s 1s/step - loss: 299.6033 - mse_known: 0.0303 - mse_unknown: 0.0784 - lr: 0.0010
Epoch 2/100
10/10 [==============================] - 12s 1s/step - loss: 299.5389 - mse_known: 0.0303 - mse_unknown: 0.0784 - lr: 0.0010
Epoch 3/100
10/10 [==============================] - 14s 1s/step - loss: 299.4746 - mse_known: 0.0302 - mse_unknown: 0.0784 - lr: 0.0010
Epoch 4/100
10/10 [==============================] - 19s 2s/step - loss: 299.4099 - mse_known: 0.0302 - mse_unknown: 0.0784 - lr: 0.0010
Epoch 5/100
10/10 [==============================] - 18s 2s/step - loss: 299.3453 - mse_known: 0.0302 - mse_unknown: 0.0784 - lr: 0.0010
Epoch 6/100
10/10 [================

Learned Variables: [14.70948, 14.670912, 14.569287, 14.35286]


###########################################################################################
################################### Sensing Ratio: 5.0% ###################################
###########################################################################################


Epoch 1/100
10/10 [==============================] - 21s 2s/step - loss: 282.4632 - mse_known: 0.0297 - mse_unknown: 0.0833 - lr: 0.0010
Epoch 2/100
10/10 [==============================] - 23s 2s/step - loss: 282.4028 - mse_known: 0.0297 - mse_unknown: 0.0833 - lr: 0.0010
Epoch 3/100
10/10 [==============================] - 24s 2s/step - loss: 282.3424 - mse_known: 0.0297 - mse_unknown: 0.0833 - lr: 0.0010
Epoch 4/100
10/10 [==============================] - 17s 2s/step - loss: 282.2818 - mse_known: 0.0297 - mse_unknown: 0.0833 - lr: 0.0010
Epoch 5/100
10/10 [==============================] - 13s 1s/step - loss: 282.2213 - mse_known: 0.0297 - mse_un

Learned Variables: [14.712183, 14.678728, 14.584199, 14.378279]


###########################################################################################
################################### Sensing Ratio: 10.0% ###################################
###########################################################################################


Epoch 1/100
10/10 [==============================] - 21s 2s/step - loss: 262.2551 - mse_known: 0.0291 - mse_unknown: 0.0841 - lr: 0.0010
Epoch 2/100
10/10 [==============================] - 23s 2s/step - loss: 262.2002 - mse_known: 0.0291 - mse_unknown: 0.0841 - lr: 0.0010
Epoch 3/100
10/10 [==============================] - 22s 2s/step - loss: 262.1451 - mse_known: 0.0291 - mse_unknown: 0.0841 - lr: 0.0010
Epoch 4/100
10/10 [==============================] - 22s 2s/step - loss: 262.0906 - mse_known: 0.0291 - mse_unknown: 0.0841 - lr: 0.0010
Epoch 5/100
10/10 [==============================] - 22s 2s/step - loss: 262.0354 - mse_known: 0.0290 - mse

Learned Variables: [14.715343, 14.688471, 14.6040745, 14.417722]


###########################################################################################
################################### Sensing Ratio: 15.0% ###################################
###########################################################################################


Epoch 1/100
10/10 [==============================] - 19s 2s/step - loss: 242.4761 - mse_known: 0.0284 - mse_unknown: 0.0862 - lr: 0.0010
Epoch 2/100
10/10 [==============================] - 24s 2s/step - loss: 242.4268 - mse_known: 0.0284 - mse_unknown: 0.0862 - lr: 0.0010
Epoch 3/100
10/10 [==============================] - 22s 2s/step - loss: 242.3774 - mse_known: 0.0284 - mse_unknown: 0.0862 - lr: 0.0010
Epoch 4/100
10/10 [==============================] - 20s 2s/step - loss: 242.3281 - mse_known: 0.0284 - mse_unknown: 0.0862 - lr: 0.0010
Epoch 5/100
10/10 [==============================] - 21s 2s/step - loss: 242.2790 - mse_known: 0.0284 - ms

Learned Variables: [14.71937, 14.699206, 14.624885, 14.45782]


###########################################################################################
################################### Sensing Ratio: 20.0% ###################################
###########################################################################################


Epoch 1/100
10/10 [==============================] - 22s 2s/step - loss: 222.9268 - mse_known: 0.0277 - mse_unknown: 0.0885 - lr: 0.0010
Epoch 2/100
10/10 [==============================] - 23s 2s/step - loss: 222.8827 - mse_known: 0.0277 - mse_unknown: 0.0885 - lr: 0.0010
Epoch 3/100
10/10 [==============================] - 23s 2s/step - loss: 222.8383 - mse_known: 0.0277 - mse_unknown: 0.0885 - lr: 0.0010
Epoch 4/100
10/10 [==============================] - 24s 2s/step - loss: 222.7940 - mse_known: 0.0277 - mse_unknown: 0.0885 - lr: 0.0010
Epoch 5/100
10/10 [==============================] - 25s 2s/step - loss: 222.7495 - mse_known: 0.0277 - mse_u

...

Epoch 83/100
10/10 [==============================] - 8s 830ms/step - loss: 851.5704 - mse_known: 0.0998 - mse_unknown: 0.1505 - lr: 5.7091e-05
Epoch 84/100
10/10 [==============================] - 8s 827ms/step - loss: 851.4727 - mse_known: 0.0998 - mse_unknown: 0.1505 - lr: 5.4895e-05
Epoch 85/100
10/10 [==============================] - 8s 829ms/step - loss: 851.3813 - mse_known: 0.0997 - mse_unknown: 0.1505 - lr: 5.2784e-05
Epoch 86/100
10/10 [==============================] - 8s 837ms/step - loss: 851.2211 - mse_known: 0.0997 - mse_unknown: 0.1505 - lr: 5.0754e-05
Epoch 87/100
10/10 [==============================] - 8s 845ms/step - loss: 851.0585 - mse_known: 0.0997 - mse_unknown: 0.1505 - lr: 4.8801e-05
Epoch 88/100
10/10 [==============================] - 8s 828ms/step - loss: 850.9933 - mse_known: 0.0997 - mse_unknown: 0.1505 - lr: 4.6924e-05
Epoch 89/100
10/10 [==============================] - 8s 826ms/step - loss: 850.8971 - mse_known: 0.0997 - mse_unknown: 0.1505 - lr: 4.5

10/10 [==============================] - 8s 842ms/step - loss: 845.1066 - mse_known: 0.1050 - mse_unknown: 0.1567 - lr: 3.7512e-04
Epoch 36/100
10/10 [==============================] - 8s 826ms/step - loss: 844.2300 - mse_known: 0.1049 - mse_unknown: 0.1566 - lr: 3.6069e-04
Epoch 37/100
10/10 [==============================] - 8s 848ms/step - loss: 843.3293 - mse_known: 0.1048 - mse_unknown: 0.1565 - lr: 3.4682e-04
Epoch 38/100
10/10 [==============================] - 8s 837ms/step - loss: 842.5597 - mse_known: 0.1047 - mse_unknown: 0.1565 - lr: 3.3348e-04
Epoch 39/100
10/10 [==============================] - 9s 875ms/step - loss: 841.9098 - mse_known: 0.1046 - mse_unknown: 0.1564 - lr: 3.2065e-04
Epoch 40/100
10/10 [==============================] - 8s 833ms/step - loss: 841.0826 - mse_known: 0.1045 - mse_unknown: 0.1563 - lr: 3.0832e-04
Epoch 41/100
10/10 [==============================] - 8s 843ms/step - loss: 840.4048 - mse_known: 0.1044 - mse_unknown: 0.1562 - lr: 2.9646e-04
Epoch

Epoch 92/100
10/10 [==============================] - 8s 829ms/step - loss: 825.1730 - mse_known: 0.1025 - mse_unknown: 0.1547 - lr: 4.0111e-05
Epoch 93/100
10/10 [==============================] - 9s 858ms/step - loss: 824.9410 - mse_known: 0.1025 - mse_unknown: 0.1547 - lr: 3.8568e-05
Epoch 94/100
10/10 [==============================] - 8s 836ms/step - loss: 824.9039 - mse_known: 0.1025 - mse_unknown: 0.1547 - lr: 3.7085e-05
Epoch 95/100
10/10 [==============================] - 8s 830ms/step - loss: 824.7828 - mse_known: 0.1024 - mse_unknown: 0.1547 - lr: 3.5659e-05
Epoch 96/100
10/10 [==============================] - 9s 885ms/step - loss: 824.6789 - mse_known: 0.1024 - mse_unknown: 0.1547 - lr: 3.4287e-05
Epoch 97/100
10/10 [==============================] - 8s 835ms/step - loss: 824.6688 - mse_known: 0.1024 - mse_unknown: 0.1547 - lr: 3.2969e-05
Epoch 98/100
10/10 [==============================] - 7s 699ms/step - loss: 824.5468 - mse_known: 0.1024 - mse_unknown: 0.1547 - lr: 3.1

10/10 [==============================] - 7s 694ms/step - loss: 810.0000 - mse_known: 0.1071 - mse_unknown: 0.1601 - lr: 2.6355e-04
Epoch 45/100
10/10 [==============================] - 7s 691ms/step - loss: 809.4022 - mse_known: 0.1070 - mse_unknown: 0.1600 - lr: 2.5342e-04
Epoch 46/100
10/10 [==============================] - 7s 704ms/step - loss: 808.8148 - mse_known: 0.1069 - mse_unknown: 0.1600 - lr: 2.4367e-04
Epoch 47/100
10/10 [==============================] - 7s 712ms/step - loss: 808.3497 - mse_known: 0.1068 - mse_unknown: 0.1599 - lr: 2.3430e-04
Epoch 48/100
10/10 [==============================] - 7s 695ms/step - loss: 807.7957 - mse_known: 0.1068 - mse_unknown: 0.1599 - lr: 2.2529e-04
Epoch 49/100
10/10 [==============================] - 7s 693ms/step - loss: 807.3367 - mse_known: 0.1067 - mse_unknown: 0.1598 - lr: 2.1662e-04
Epoch 50/100
10/10 [==============================] - 7s 699ms/step - loss: 806.8751 - mse_known: 0.1067 - mse_unknown: 0.1598 - lr: 2.0829e-04
Epoch

Model: "EM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 4
Trainable params: 4
Non-trainable params: 0
_________________________________________________________________
Learned Variables: [12.044343, 14.030416, 13.879012, 11.237519]


###########################################################################################
################################### Sensing Ratio: 30.0% ###################################
###########################################################################################


Epoch 1/100
10/10 [==============================] - 7s 700ms/step - loss: 835.7999 - mse_known: 0.1180 - mse_unknown: 0.1688 - lr: 0.0010
Epoch 2/100
10/10 [==============================] - 7s 698ms/step - loss: 834.1318 - mse_known: 0.1177 - mse_unknown: 0.1687 - lr: 0.0010
Epoch 3/100
10/10 [==============================] - 7s 697ms/step - loss: 832.3475 - mse_known: 0.1175 - m

In [16]:
for i, rem in enumerate(sensing_ratio):
    print(f"For {int(100 * rem)}% sensing ratio: {learned_variables[i]}")

For 1% sensing ratio: [14.70948, 14.670912, 14.569287, 14.35286]
For 5% sensing ratio: [14.712183, 14.678728, 14.584199, 14.378279]
For 10% sensing ratio: [14.715343, 14.688471, 14.6040745, 14.417722]
For 15% sensing ratio: [14.71937, 14.699206, 14.624885, 14.45782]
For 20% sensing ratio: [14.723341, 14.710116, 14.645437, 14.494876]
For 25% sensing ratio: [14.729137, 14.72241, 14.666371, 14.528871]
For 30% sensing ratio: [14.736987, 14.735822, 14.68679, 14.55755]
For 35% sensing ratio: [14.747079, 14.751162, 14.708162, 14.583054]
For 40% sensing ratio: [14.758982, 14.767932, 14.729951, 14.60549]
For 45% sensing ratio: [14.769721, 14.783657, 14.750447, 14.626058]
For 50% sensing ratio: [14.782383, 14.800355, 14.771268, 14.647086]


In [17]:
np.save('./Outputs/eldar_matic_mse_known.npy', mse_known_list)
np.save('./Outputs/eldar_matic_mse_unknown.npy', mse_unknown_list)